In [1]:
import scipy.io as scio
import netCDF4 as nc
import numpy as np
from Load_data import Data_from_nc
import xarray as xr
import math
import gc

def get_data_from_nc(file, variable):
    file_obj = nc.Dataset(file)
    data = file_obj.variables[variable]
    var_data = np.array(data)
    var_data = var_data[:, :, :]
    var_data = np.squeeze(var_data)
    return var_data
    
def Apparent_temperature_calcu(TSA, RH, U10):
    T_c = TSA - 273.15
    T_f = T_c * 1.8 + 32
    e_s = np.exp(53.67957 - 6743.769 / TSA - 4.8451 * np.log(TSA)) * 100
    e_RH = (RH / 100) * e_s
    AT = T_c + 3.3 * e_RH / 1000 - 0.7 * U10 - 4
    return AT

def Heat_index_calcu(TSA, RH):
    T_c = TSA - 273.15
    T_f = T_c * 1.8 + 32
    HI = -42.379 + 2.04901523 * T_f + 10.14333127 * RH - 0.22475541 * T_f * RH -6.83783 * 0.001 * np.square(T_f) - 5.481717 * 0.01 * np.square(RH) + 1.22874 * 0.001 * np.square(T_f) * RH + 8.5282 * 0.0001 * T_f * np.square(RH) - 1.99 * 0.000001 * np.square(T_f) * np.square(RH)
    return HI

def Humidex_calcu(TSA, RH):
    T_c = TSA - 273.15
    T_f = T_c * 1.8 + 32
    e_s = np.exp(53.67957 - 6743.769 / TSA - 4.8451 * np.log(TSA)) * 100
    e_RH = (RH / 100) * e_s
    HUMIDEX = T_c + 5/9 * (e_RH / 100 - 10)
    return HUMIDEX

def Wet_bulb_temperature_calcu(TSA, RH):
    T_c = TSA - 273.15
    T_w = T_c * np.arctan(0.151977 * np.sqrt(RH + 8.313659)) + np.arctan(T_c + RH) - np.arctan(RH - 1.676331) + 0.00391838 * np.power(RH, 3/2) * np.arctan(0.023101 * RH) - 4.68035
    return T_w

def Temperature_humidity_index_for_comfort_calcu(TSA, RH):
    T_c = TSA - 273.15
    T_w = Wet_bulb_temperature_calcu(TSA, RH)
    THIC = 0.72 * T_w + 0.72 * T_c + 40.6
    return THIC

def Temperature_humidity_index_for_physiology_calcu(TSA, RH):
    T_c = TSA - 273.15
    T_w = Wet_bulb_temperature_calcu(TSA, RH)
    THIP = 0.63 * T_w + 1.17 * T_c + 32
    return THIP

def Discomfort_index_calcu(TSA, RH):
    T_c = TSA - 273.15
    T_w = Wet_bulb_temperature_calcu(TSA, RH)
    DI = 0.5 * T_w + 0.5 * T_c
    return DI

def Swamp_temperature_calcu(TSA, RH, eta):
    T_c = TSA - 273.15
    T_w = Wet_bulb_temperature_calcu(TSA, RH)
    T_s = T_c - eta / 100 * (T_c - T_w)
    return T_s

# Here we calculate the metrics based on the fifth ensemble member of tranirr. For other ensemble members adjustments are needed

for year in range(1901, 2015):
    TSA_daily = get_data_from_nc('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/CNRM/CNRM-CM6-1_hist-irr_r5i1p1f2/tasmax/tasmax_day_CNRM-CM6-1_hist-irr_r5i1p1f2_gr_19010101-20141231.nc_' + str(year) + '.nc_0.9x1.25', 'tasmax')
    RH2M_daily = get_data_from_nc('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/CNRM/CNRM-CM6-1_hist-irr_r5i1p1f2/hursmin/hursmin_day_CNRM-CM6-1_hist-irr_r5i1p1f2_gr_19010101-20141231.nc_' + str(year) + '.nc_0.9x1.25', 'hursmin')


    TSA_daily[TSA_daily > 1000] = np.nan
    scio.savemat("/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/CNRM/CNRM-CM6-1_hist-irr_r5i1p1f2/TSA_" + \
                 str(year) + "_3_hourly_" + exp + "_0.9x1.25.mat", {'TSA': TSA_daily})

    HUMIDEX_daily = Humidex_calcu(TSA_daily, RH2M_daily)
    scio.savemat("/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/CNRM/CNRM-CM6-1_hist-irr_r5i1p1f2/HUMIDEX_" + \
                 str(year) + "_3_hourly_" + exp + "_0.9x1.25.mat", {'HUMIDEX': HUMIDEX_daily})
    del HUMIDEX_daily
    gc.collect()
    T_w_daily = Wet_bulb_temperature_calcu(TSA_daily, RH2M_daily)
    scio.savemat("/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/CNRM/CNRM-CM6-1_hist-irr_r5i1p1f2/T_w_" + \
                 str(year) + "_3_hourly_" + exp + "_0.9x1.25.mat", {'T_w': T_w_daily})
    del T_w_daily
    gc.collect()
    THIC_daily = Temperature_humidity_index_for_comfort_calcu(TSA_daily, RH2M_daily)
    scio.savemat("/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/CNRM/CNRM-CM6-1_hist-irr_r5i1p1f2/THIC_" + \
                 str(year) + "_3_hourly_" + exp + "_0.9x1.25.mat", {'THIC': THIC_daily})
    del THIC_daily
    gc.collect()
    THIP_daily = Temperature_humidity_index_for_physiology_calcu(TSA_daily, RH2M_daily)
    scio.savemat("/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/CNRM/CNRM-CM6-1_hist-irr_r5i1p1f2/THIP_" + \
                 str(year) + "_3_hourly_" + exp + "_0.9x1.25.mat", {'THIP': THIP_daily})
    del THIP_daily
    gc.collect()
    DI_daily = Discomfort_index_calcu(TSA_daily, RH2M_daily)
    scio.savemat("/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/CNRM/CNRM-CM6-1_hist-irr_r5i1p1f2/DI_" + \
                 str(year) + "_3_hourly_" + exp + "_0.9x1.25.mat", {'DI': DI_daily})
    del DI_daily
    gc.collect()
    T_s_65_daily = Swamp_temperature_calcu(TSA_daily, RH2M_daily, 65)
    scio.savemat("/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/CNRM/CNRM-CM6-1_hist-irr_r5i1p1f2/T_s_65_" + \
                 str(year) + "_3_hourly_" + exp + "_0.9x1.25.mat", {'T_s_65': T_s_65_daily})
    del T_s_65_daily
    gc.collect()
    T_s_80_daily = Swamp_temperature_calcu(TSA_daily, RH2M_daily, 80)
    scio.savemat("/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/CNRM/CNRM-CM6-1_hist-irr_r5i1p1f2/T_s_80_" + \
                 str(year) + "_3_hourly_" + exp + "_0.9x1.25.mat", {'T_s_80': T_s_80_daily})
    del T_s_80_daily
    gc.collect()

/tmp/ipykernel_1815730/3095593529.py:43: RuntimeWarning: invalid value encountered in power
  T_w = T_c * np.arctan(0.151977 * np.sqrt(RH + 8.313659)) + np.arctan(T_c + RH) - np.arctan(RH - 1.676331) + 0.00391838 * np.power(RH, 3/2) * np.arctan(0.023101 * RH) - 4.68035
